In [1]:
import pandas as pd
import altair as alt

In [2]:
dados_municipais_campina = pd.read_csv("data\\dados_municipais_cg.csv", encoding='utf-8', index_col='data')
dados_federais_campina = pd.read_csv("data\\dados_federais_vacinacao_cg.csv", encoding='UTF-8', error_bad_lines=False)

C:\Users\perei\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


# Processando os dados sobre Aplicação de vacinas por dose e por dia

In [3]:
# Função que cria os dataframes e 'seta' a coluna 'data' como index
def criar_dataframe(dataframe, nome_coluna):
    df = pd.DataFrame({
        'data': dataframe.index.tolist(),
        nome_coluna: dataframe.values.tolist()
    })

    # Setando a coluna 'data' como índice do DataFrame 
    df.set_index('data', inplace=True)
    return df

## Dados Federais

In [4]:
# Separando os dados de aplicação da vacina por dose da base federal.

# Primeira Dose
df_campina1Dose = dados_federais_campina.loc[dados_federais_campina['vacina_descricao_dose'] == '1ª Dose']
df_campina1Dose = df_campina1Dose.loc[df_campina1Dose['vacina_codigo'] != 88]
df_1DosePorDia = df_campina1Dose['vacina_dataaplicacao'].value_counts()

# Segunda Dose
df_campina2Dose = dados_federais_campina.loc[dados_federais_campina['vacina_descricao_dose'] == '2ª Dose']
df_campina2Dose = df_campina2Dose.loc[df_campina2Dose['vacina_codigo'] != 88]
df_2DosePorDia = df_campina2Dose['vacina_dataaplicacao'].value_counts()

# Dose Única
df_campinaDoseUnica = dados_federais_campina.loc[dados_federais_campina['vacina_codigo'] == 88]
df_DoseUnicaPorDia = df_campinaDoseUnica['vacina_dataaplicacao'].value_counts()

# Ordenando os dados pela data
df_1DosePorDia     = df_1DosePorDia.sort_index()
df_2DosePorDia     = df_2DosePorDia.sort_index()
df_DoseUnicaPorDia = df_DoseUnicaPorDia.sort_index()

### Primeira Dose

In [5]:
# Criando um DataFrame com os dados para o altair
df_1DosePorDia = criar_dataframe(df_1DosePorDia, 'aplic_1DosePorDia')

# Criando uma nova coluna no DataFrame referente a média movel.
df_1DosePorDia['media_movel'] = df_1DosePorDia.aplic_1DosePorDia.rolling(7).mean()

### Segunda Dose

In [6]:
# Criando um DataFrame com os dados para o altair
df_2DosePorDia = criar_dataframe(df_2DosePorDia, 'aplic_2DosePorDia')

# Criando uma nova coluna no DataFrame referente a média movel.
df_2DosePorDia['media_movel'] = df_2DosePorDia.aplic_2DosePorDia.rolling(7).mean()

### Dose Única

In [7]:
# Criando um DataFrame com os dados para o altair
df_DoseUnicaPorDia = criar_dataframe(df_DoseUnicaPorDia, 'aplic_DoseUnicaPorDia')

# Criando uma nova coluna no DataFrame referente a média movel.
df_DoseUnicaPorDia['media_movel'] = df_DoseUnicaPorDia.aplic_DoseUnicaPorDia.rolling(7).mean()

## Dados Municipais

In [8]:
# Separando os dados de aplicação da vacina por dose da base municipal.

# Primeira Dose
dm_1DosePorDia = dados_municipais_campina['vacinadosDose1Dia']

# Segunda Dose
dm_2DosePorDia = dados_municipais_campina['vacinadosDose2Dia']

# Dose Única
dm_DoseUnicaPorDia = dados_municipais_campina['vacinadosDoseUnicaDia']


### Primeira Dose

In [9]:
# Criando um DataFrame com os dados para o altair
dm_1DosePorDia = criar_dataframe(dm_1DosePorDia, 'aplic_1DosePorDia')

# Criando uma nova coluna no DataFrame referente a média movel.
dm_1DosePorDia['media_movel'] = dm_1DosePorDia.aplic_1DosePorDia.rolling(7).mean()

### Segunda Dose

In [10]:
# Criando um DataFrame com os dados para o altair
dm_2DosePorDia = criar_dataframe(dm_2DosePorDia, 'aplic_2DosePorDia')

# Criando uma nova coluna no DataFrame referente a média movel.
dm_2DosePorDia['media_movel'] = dm_2DosePorDia.aplic_2DosePorDia.rolling(7).mean()

### Dose Única

In [11]:
# Criando um DataFrame com os dados para o altair
dm_DoseUnicaPorDia = criar_dataframe(dm_DoseUnicaPorDia, 'aplic_DoseUnicaPorDia')

# Criando uma nova coluna no DataFrame referente a média movel.
dm_DoseUnicaPorDia['media_movel'] = dm_DoseUnicaPorDia.aplic_DoseUnicaPorDia.rolling(7).mean()

## Excluindo as datas não contidas nos DataFrames (Federal e Municipal)

In [12]:
# Função que exclui os elementos (pelo index) do DataFrame que não estão em outro DataFrame
def exclui_nao_contidos(df_conjunto, df_subconjunto):
    for i in df_conjunto.index:
        if i not in df_subconjunto.index:
            df_conjunto = df_conjunto.drop(i)
    
    return df_conjunto

### Primeira Dose

In [13]:
# Excluindo as datas que estão nos dados federais mas não estão nos dados municipais
df_1DosePorDia = exclui_nao_contidos(df_1DosePorDia, dm_1DosePorDia)

# Excluindo as datas que estão nos dados municipais mas não estão nos dados federais
dm_1DosePorDia = exclui_nao_contidos(dm_1DosePorDia, df_1DosePorDia)

### Segunda Dose

In [14]:
# Excluindo as datas que estão nos dados federais mas não estão nos dados municipais
df_2DosePorDia = exclui_nao_contidos(df_2DosePorDia, dm_2DosePorDia)

# Excluindo as datas que estão nos dados municipais mas não estão nos dados federais
dm_2DosePorDia = exclui_nao_contidos(dm_2DosePorDia, df_2DosePorDia)

### Dose Única

In [15]:
# Excluindo as datas que estão nos dados federais mas não estão nos dados municipais
df_DoseUnicaPorDia = exclui_nao_contidos(df_DoseUnicaPorDia, dm_DoseUnicaPorDia)

# Excluindo as datas que estão nos dados municipais mas não estão nos dados federais
dm_DoseUnicaPorDia = exclui_nao_contidos(dm_DoseUnicaPorDia, df_DoseUnicaPorDia)

# Estatística descritiva dos Dados

In [16]:
def estatistica_descritiva(df, dm, col_name):
    print("[DF] -> Dados Federais\n[DM] -> Dados Municipais")
    print("Intervalo de 2021-06-01 até 2021-08-31")
    print("     média     mediana  variância   DP")
    print("[DF] %.2f    %.2f   %.2f    %.2f" % (df[col_name].mean(), 
                                                df[col_name].median(), 
                                                df[col_name].var(), 
                                                df[col_name].std()))
    
    print("[DM] %.2f    %.2f   %.2f    %.2f" % (dm[col_name].mean(), 
                                                dm[col_name].median(), 
                                                dm[col_name].var(), 
                                                dm[col_name].std()))

## Primeira Dose

In [17]:
estatistica_descritiva(df_1DosePorDia, dm_1DosePorDia, 'aplic_1DosePorDia')

[DF] -> Dados Federais
[DM] -> Dados Municipais
Intervalo de 2021-06-01 até 2021-08-31
     média     mediana  variância   DP
[DF] 1718.57    948.00   4251727.73    2061.97
[DM] 1891.91    720.00   7039410.08    2653.19


## Segunda Dose

In [18]:
estatistica_descritiva(df_2DosePorDia, dm_2DosePorDia, 'aplic_2DosePorDia')

[DF] -> Dados Federais
[DM] -> Dados Municipais
Intervalo de 2021-06-01 até 2021-08-31
     média     mediana  variância   DP
[DF] 668.81    494.50   352572.02    593.78
[DM] 669.89    575.50   442754.50    665.40


## Dose Única

In [19]:
estatistica_descritiva(df_DoseUnicaPorDia, dm_DoseUnicaPorDia, 'aplic_DoseUnicaPorDia')

[DF] -> Dados Federais
[DM] -> Dados Municipais
Intervalo de 2021-06-01 até 2021-08-31
     média     mediana  variância   DP
[DF] 273.26    10.00   324122.11    569.32
[DM] 271.52    0.00   536455.17    732.43


## Transformando os DataFrame do formato wide para o formato long

In [20]:
# Função que transforma os DataFrames do formato wide para o formato long
def to_longFormat(dt_federal, dt_municipal, nome_coluna):
    long_df = pd.DataFrame({
    'data': dt_federal.index.tolist(),
    'Federal': dt_federal[nome_coluna].tolist(),
    'Municipal': dt_municipal[nome_coluna].tolist()
    }).melt(
        id_vars=['data'], 
        value_vars=['Federal', 'Municipal'],
        var_name='fonte_dados', 
        value_name='aplic_por_dia'
    )
    return long_df

### Primeira Dose

In [21]:
# Transformando os DataFrames da aplicação da primeira dose
vacinados1DosePorDia = to_longFormat(df_1DosePorDia, dm_1DosePorDia,'aplic_1DosePorDia')

# Transformando os DataFrames da média movel da aplicação da primeira dose
vacinados1DosePorDiaMedMovel = to_longFormat(df_1DosePorDia, dm_1DosePorDia,'media_movel')

### Segunda Dose

In [22]:
# Transformando os DataFrames da aplicação da segunda dose
vacinados2DosePorDia = to_longFormat(df_2DosePorDia, dm_2DosePorDia,'aplic_2DosePorDia')

# Transformando os DataFrames da média movel da aplicação da primeira dose
vacinados2DosePorDiaMedMovel = to_longFormat(df_2DosePorDia, dm_2DosePorDia,'media_movel')

### Dose Única

In [23]:
# Transformando os DataFrames da aplicação da dose única
vacinadosDoseUnicaPorDia = to_longFormat(df_DoseUnicaPorDia, dm_DoseUnicaPorDia,'aplic_DoseUnicaPorDia')

# Transformando os DataFrames da média movel da aplicação da primeira dose
vacinadosDoseUnicaPorDiaMedMovel = to_longFormat(df_DoseUnicaPorDia, dm_DoseUnicaPorDia,'media_movel')

# Comparando os dados Federais e Municipais

In [24]:
# Função que cria os graficos de linhas (mark_line) no altair
def criar_mark_line(dataframe, titulo, pontos):
    mark_line = alt.Chart(dataframe).mark_line(point=pontos).encode(
                    alt.X('data:T', title='Data'),
                    alt.Y('aplic_por_dia:Q', title='Nº de Vacinados'),
                    alt.Color('fonte_dados:N', title='Base dos Dados')
                ).properties(
                    width=650,
                    height=400,
                    title=titulo
                )
    return mark_line

In [25]:
# Função que cria os graficos de barras (mark_bar) no altair
def criar_mark_bar(dataframe, titulo):
    marak_bar = alt.Chart(dataframe).mark_bar(size=50).encode(
                    alt.X('base:N', title='Base dos Dados'),
                    alt.Y('total:Q', title='Nº Total de Vacinados'),
                    alt.Color('base:N', title='Base de Dados')
                ).properties(
                    width=250,
                    title=titulo
                )
    return marak_bar

### Aplicações da 1º Dose por Dia

In [26]:
criar_mark_line(vacinados1DosePorDia, 'Quantidade de vacinados com 1º Dose por Dia', False)

alt.Chart(...)

### Média Movel (7 dias) da Aplicação da 1º Dose

In [27]:
criar_mark_line(vacinados1DosePorDiaMedMovel, 'Média movel da quantidade de vacinados com 1º Dose por Dia', True)

alt.Chart(...)

### Total Aplicações da 1º Dose

In [28]:
df_total1Dose = df_1DosePorDia['aplic_1DosePorDia'].sum()
dm_total1Dose = dm_1DosePorDia['aplic_1DosePorDia'].sum()

In [29]:
total1Dose = pd.DataFrame({
    'base': ['Federal', 'Municipal'],
    'total': [df_total1Dose, dm_total1Dose]
})

In [30]:
criar_mark_bar(total1Dose, 'Total de aplicações da Primeira Dose')

alt.Chart(...)

### Aplicações da 2º Dose por Dia

In [31]:
criar_mark_line(vacinados2DosePorDia, 'Quantidade de vacinados com 2º Dose por Dia', False)

alt.Chart(...)

### Média Movel (7 dias) da Aplicação da 2º Dose

In [32]:
criar_mark_line(vacinados2DosePorDiaMedMovel, 'Média movel da quantidade de vacinados com 2º Dose por Dia', True)


alt.Chart(...)

### Total Aplicações da 2º Dose

In [33]:
df_total2Dose = df_2DosePorDia['aplic_2DosePorDia'].sum()
dm_total2Dose = dm_2DosePorDia['aplic_2DosePorDia'].sum()

In [34]:
total2Dose = pd.DataFrame({
    'base': ['Federal', 'Municipal'],
    'total': [df_total2Dose, dm_total2Dose]
})

In [35]:
criar_mark_bar(total2Dose, 'Total de aplicações da Segunda Dose')

alt.Chart(...)

### Aplicações da Dose Única por Dia

In [46]:
criar_mark_line(vacinadosDoseUnicaPorDia, 'Quantidade de vacinados com Dose Única por Dia', False)

alt.Chart(...)

### Média Movel (7 dias) da Aplicação da Dose Única

In [47]:
criar_mark_line(vacinadosDoseUnicaPorDiaMedMovel, 'Média movel da quantidade de vacinados com Dose Única por Dia', True)

alt.Chart(...)

### Total Aplicações da Dose Única

In [48]:
df_totalDoseUnica = df_DoseUnicaPorDia['aplic_DoseUnicaPorDia'].sum()
dm_totalDoseUnica = dm_DoseUnicaPorDia['aplic_DoseUnicaPorDia'].sum()

In [49]:
totalDoseUnica = pd.DataFrame({
    'base': ['Federal', 'Municipal'],
    'total': [df_totalDoseUnica, dm_totalDoseUnica]
})

In [50]:
criar_mark_bar(totalDoseUnica, 'Total de aplicações da Dose Única')

alt.Chart(...)

### Total aplicações da vacinas contra o covid-19.

In [51]:
df_totalAplic = df_total1Dose + df_total2Dose + df_totalDoseUnica
dm_totalAplic = dm_total1Dose + dm_total2Dose + dm_totalDoseUnica

In [52]:
vacinadosTotal = pd.DataFrame({
    'base': ['Federal', 'Municipal'],
    'total': [df_totalAplic, dm_totalAplic]
})

In [53]:
criar_mark_bar(vacinadosTotal, 'Total de Doses Aplicadas')

alt.Chart(...)